In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pickle
from tqdm import tqdm

In [2]:
from geopy.geocoders import Nominatim
# geolocalização
geolocator = Nominatim(user_agent="myApp", timeout=5)

In [3]:
with open('data/houses.pkl', 'rb') as f:
    obj = pickle.load(f)

In [5]:
df = pd.DataFrame(obj.values())

Ajustando o valor do imóvel

In [6]:
df = pd.concat([df, pd.DataFrame(df.price.to_dict().values(), columns=['rent', 'Condomínio', 'Preço c/ condomínio','IPTU'])], axis=1)

Ajustando o banheiro, removendo a lista

In [7]:
df.bathrooms = df.bathrooms.map(lambda x: int(x[0]) if isinstance(x, list) and (not not x) else 0)

In [8]:
df.iloc[0:3].address.apply(geolocator.geocode).apply(lambda x: pd.Series([x.latitude, x.longitude], index=['location_lat', 'location_long']))

,location_lat,location_long
0,-23.650586,-46.718644
1,-23.723508,-46.693460
2,-21.132831,-51.102942


In [9]:
df.drop(columns=['total_price', 'area', 'description', 'transport', 'publisher_name', 'code', 'type'], inplace=True)

In [10]:
df['Condomínio'] = df['Condomínio'].str.replace('R$ ', '', regex=False)

In [11]:
df['Condomínio'] = df['Condomínio'].str.strip()

In [12]:
df['Condomínio'] = df['Condomínio'].replace({'Não informado': 0, 'Preço c/ condomínio':0, 'IPTU':'0'})

In [13]:
df['Condomínio'] = df['Condomínio'].str.replace('.', '', regex=False)

In [14]:
df['Condomínio'] = df['Condomínio'].fillna(0)

In [15]:
df['Condomínio'] = df['Condomínio'].astype(int)

In [16]:
df['Preço c/ condomínio'] = df['Preço c/ condomínio'].replace({'R$ ':''}, regex=False).fillna(0)

In [17]:
df['Preço c/ condomínio'] = df['Preço c/ condomínio'].str.replace('R$ ','', regex=False).str.replace('.', '', regex=False).fillna(0).astype(int)

In [18]:
df['IPTU'] = df['IPTU'].str.replace('R$ ','', regex=False).str.replace('.', '', regex=False).replace({' Não informado ':0}, regex=False).fillna(0).astype(int)

In [41]:
df.drop(columns=['price'], inplace=True)

Endereço e amenities

Considerar os bairos.

In [44]:
df.shape

(1826, 11)

In [48]:
df.sample(frac=0.05)#pega 5% do dataframe

,address,url,bedrooms,bathrooms,parking,amenities,title,rent,Condomínio,Preço c/ condomínio,IPTU
1449,None,https://www.vivareal.com.br/imovel/apartamento...,NaN,0,NaN,[],None,NaN,0,0,0
844,"Vila Leonor, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,2.0,21,2.0,"[Piscina, Mobiliado, Churrasqueira, Elevador, ...",Apartamento com 2 Quartos e 2 banheiros para...,1850.0,780,2630,174
940,"Avenida Ramalho Ortigão, 356 - Vila Gumercindo...",https://www.vivareal.com.br/imovel/apartamento...,2.0,21,2.0,"[Piscina, Mobiliado, Elevador, Condomínio fech...",Apartamento com 2 Quartos e 2 banheiros para...,4500.0,920,5420,349
950,"Rua Kansas - Brooklin, São Paulo - SP",https://www.vivareal.com.br/imovel/sobrado-3-q...,3.0,31,1.0,[],"Sobrado com 3 Quartos e 3 banheiros à Venda,...",2000000.0,0,0,0
547,"Rua das Fiandeiras - Vila Olímpia, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,1.0,11,1.0,"[Piscina, Churrasqueira, Elevador, Varanda, Ac...",Apartamento com 1 Quarto e 1 banheiro para A...,2700.0,860,3560,170
...,...,...,...,...,...,...,...,...,...,...,...
474,"Rua Castrinópolis - Vila Santa Maria, São Paul...",https://www.vivareal.com.br/imovel/casa-1-quar...,1.0,1,NaN,[],"Casa com 1 Quarto e 1 banheiro para Alugar, ...",1000.0,0,0,0
489,"Rua Carneiro Leão, 385 - Brás, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,2.0,1,1.0,"[Churrasqueira, Elevador, Condomínio fechado, ...",Apartamento com 2 Quartos e 1 banheiro para ...,1280.0,370,1650,80
335,None,https://www.vivareal.com.br/imovel/apartamento...,NaN,0,NaN,[],None,NaN,0,0,0
324,None,https://www.vivareal.com.br/imovel/apartamento...,NaN,0,NaN,[],None,NaN,0,0,0


In [99]:
#organizar as amenities em ordem alfabética
df.amenities = df.amenities.apply(lambda x: sorted(x))

In [103]:
# identificando todas as amenities
amenities = []
df.amenities.apply(lambda x : amenities.extend(x))
amenities = set(sorted(amenities))

In [106]:
df_amenities = pd.DataFrame(df.amenities.apply(lambda x: {_:1 for _ in x}).to_list())

In [110]:
df_amenities.fillna(0, inplace=True)

In [111]:
pd.concat([df, df_amenities], axis=1)

,address,url,bedrooms,bathrooms,parking,amenities,title,rent,Condomínio,Preço c/ condomínio,...,Drywall,Muro de vidro,Piso elevado,Poço artesiano,Muro e grade,Espaço teen,Pista de skate,Children care,Vista para lago,Centro de estética
0,"Avenida Doutor Mário Vilas Boas Rodrigues, 175...",https://www.vivareal.com.br/imovel/apartamento...,2.0,21,1.0,"[Academia, Ar-condicionado, Churrasqueira, Con...",Apartamento com 2 Quartos e 2 banheiros para...,3900.0,746,4646,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Rua Fulgêncio Vertello - Cidade Dutra, São Pau...",https://www.vivareal.com.br/imovel/sobrado-1-q...,1.0,1,NaN,[],Sobrado com 1 Quarto e 1 banheiro para Aluga...,900.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Mirandópolis, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,1.0,1,1.0,"[Academia, Churrasqueira, Piscina]",Apartamento com 1 Quarto e 1 banheiro para A...,2500.0,726,3226,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Rua dos Estudantes, 357 - Sé, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,1.0,11,NaN,"[Academia, Aceita animais, Bicicletário, Churr...",Apartamento com 1 Quarto e 1 banheiro para A...,1700.0,350,2050,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Alameda Itu, 78 - Jardins, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,1.0,11,1.0,"[Academia, Aceita animais, Acesso para deficie...",Apartamento com 1 Quarto e 1 banheiro para A...,3500.0,706,4206,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,"Rua Henri Dunant, 1066 - Chácara Santo Antônio...",https://www.vivareal.com.br/imovel/apartamento...,1.0,11,1.0,"[Academia, Aceita animais, Acesso para deficie...",Apartamento com 1 Quarto e 1 banheiro para A...,3700.0,950,4650,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1822,"Rua Jesuíno Arruda, 769 - Itaim Bibi, São Paul...",https://www.vivareal.com.br/imovel/sala-comerc...,NaN,3,3.0,[],"Sala Comercial e 3 banheiros para Alugar, 10...",4500.0,3500,8000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1823,"Rua Arandu, 205 - Brooklin, São Paulo - SP",https://www.vivareal.com.br/imovel/sala-comerc...,NaN,1,2.0,[],"Sala Comercial e 1 banheiro para Alugar, 87 ...",3900.0,870,4770,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1824,"Avenida Paulista - Bela Vista, São Paulo - SP",https://www.vivareal.com.br/imovel/ponto-comer...,NaN,1,NaN,"[Ar-condicionado, Condomínio fechado, Elevador...","Ponto Comercial e 1 banheiro para Alugar, 60...",1970.0,720,2690,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [112]:
df.to_excel('data/dados.xlsx', index=False)

## Identificar a geolocalização de cada endereço

In [19]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="MyApp")

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [21]:
tqdm.pandas()

In [22]:
geolocator = Nominatim(user_agent="myApp")

In [23]:
g=geocode('Rua General Chagas Santos - Saúde, São Paulo - SP', addressdetails =True)
g

Location(Rua General Chagas Santos, Vila do Bosque, Saúde, São Paulo, Região Imediata de São Paulo, Região Metropolitana de São Paulo, Região Geográfica Intermediária de São Paulo, São Paulo, Região Sudeste, 04138-030, Brasil, (-23.6157408, -46.6272546, 0.0))

In [33]:
g.raw

{'place_id': 281207922,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 943955517,
 'boundingbox': ['-23.6157949', '-23.6157211', '-46.6273066', '-46.6272303'],
 'lat': '-23.6157408',
 'lon': '-46.6272546',
 'display_name': 'Rua General Chagas Santos, Vila do Bosque, Saúde, São Paulo, Região Imediata de São Paulo, Região Metropolitana de São Paulo, Região Geográfica Intermediária de São Paulo, São Paulo, Região Sudeste, 04138-030, Brasil',
 'class': 'highway',
 'type': 'tertiary',
 'importance': 0.8099999999999999,
 'address': {'road': 'Rua General Chagas Santos',
  'suburb': 'Vila do Bosque',
  'city_district': 'Saúde',
  'city': 'São Paulo',
  'municipality': 'Região Imediata de São Paulo',
  'county': 'Região Metropolitana de São Paulo',
  'state_district': 'Região Geográfica Intermediária de São Paulo',
  'state': 'São Paulo',
  'ISO3166-2-lvl4': 'BR-SP',
  'region': 'Região Sudeste',
  'postcode': '04138-030',
  

In [139]:
df['geocode'] = None

In [24]:
df = pd.read_excel('data/dados.xlsx')

In [25]:
geo_dic = {}

In [34]:
df['suburb', 'city_district', 'address_class', 'postcode'] = None

In [39]:
_geo.raw

{'place_id': 167604091,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 249870997,
 'boundingbox': ['-23.65076', '-23.6503922', '-46.7187522', '-46.7186348'],
 'lat': '-23.6505856',
 'lon': '-46.7186442',
 'display_name': 'Avenida Doutor Mário Vilas Boas Rodrigues, Santo Amaro, São Paulo, Região Imediata de São Paulo, Região Metropolitana de São Paulo, Região Geográfica Intermediária de São Paulo, São Paulo, Região Sudeste, 04757-000, Brasil',
 'class': 'highway',
 'type': 'trunk_link',
 'importance': 1.075,
 'address': {'road': 'Avenida Doutor Mário Vilas Boas Rodrigues',
  'city_district': 'Santo Amaro',
  'city': 'São Paulo',
  'municipality': 'Região Imediata de São Paulo',
  'county': 'Região Metropolitana de São Paulo',
  'state_district': 'Região Geográfica Intermediária de São Paulo',
  'state': 'São Paulo',
  'ISO3166-2-lvl4': 'BR-SP',
  'region': 'Região Sudeste',
  'postcode': '04757-000',
  'country': 'Bra

In [50]:
%%time
for i, v in tqdm(df.iterrows(), total=df.shape[0]):
#     df.loc[i, 'geocode'] = 
    _geo = geocode(v['address'], addressdetails =True)
    if _geo is None:
        geo_dic[i] = False
    else:
        geo_dic[i] = {'suburb':_geo.raw.get('address', {}).get('suburb', False), 
                  'city_district':_geo.raw.get('address', {}).get('city_district', False), 
                  'address_class':_geo.raw.get('address', {}).get('address_class', False), 
                  'postcode':_geo.raw.get('address', {}).get('postcode', False)}

 17%|█████████████▊                                                                 | 319/1826 [05:20<25:15,  1.01s/it]


KeyboardInterrupt: 

In [51]:
geo_dic

{0: {'suburb': False,
  'city_district': 'Santo Amaro',
  'address_class': False,
  'postcode': '04757-000'},
 1: {'suburb': False,
  'city_district': False,
  'address_class': False,
  'postcode': '04823-280'},
 2: {'suburb': False,
  'city_district': False,
  'address_class': False,
  'postcode': '16800-000'},
 3: {'suburb': 'Itaim Paulista',
  'city_district': False,
  'address_class': False,
  'postcode': '08541-310'},
 4: False,
 5: {'suburb': 'Vila do Bosque',
  'city_district': 'Saúde',
  'address_class': False,
  'postcode': '04138-030'},
 6: {'suburb': 'Butantã',
  'city_district': 'Butantã',
  'address_class': False,
  'postcode': '05590-190'},
 7: {'suburb': 'Pinheiros',
  'city_district': 'Bairro de Pinheiros',
  'address_class': False,
  'postcode': '05423-030'},
 8: False,
 9: {'suburb': 'Ipiranga',
  'city_district': 'Ipiranga',
  'address_class': False,
  'postcode': '04209-003'},
 10: {'suburb': 'Ipiranga',
  'city_district': 'Ipiranga',
  'address_class': False,
  'po

In [179]:
df = pd.concat([df.drop(columns='geocode'), pd.DataFrame(geo_dic.items()).drop(columns=0).rename({1:'geocode'}, axis=1)], axis=1)

In [138]:
%%time
df['geocode'] = df['address'].progress_apply(geocode, addressdetails =True)#.progress_apply(lambda location: (location.latitude, location.longitude))
# df['coord'] = df['address'].progress_apply(geocode).progress_apply(lambda location: (location.latitude, location.longitude))
# df.head()

  6%|████▍                                                                          | 103/1826 [01:44<28:43,  1.00s/it]RateLimiter caught an error, retrying (0/2 tries). Called with (*('Avenida Jônia, 326 - Jardim Brasil, São Paulo - SP',), **{'addressdetails': True}).
Traceback (most recent call last):
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 387, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 383, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 1371, in getresponse
    response.begin()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 280, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "

RateLimiter swallowed an error after 2 retries. Called with (*('Avenida Jônia, 326 - Jardim Brasil, São Paulo - SP',), **{'addressdetails': True}).
Traceback (most recent call last):
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 387, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 383, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 1371, in getresponse
    response.begin()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 280, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\Alexandre\anaconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)

RateLimiter caught an error, retrying (1/2 tries). Called with (*('Americanópolis, São Paulo - SP',), **{'addressdetails': True}).
Traceback (most recent call last):
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 387, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 383, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 1371, in getresponse
    response.begin()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 280, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\Alexandre\anaconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "C:\Users

 10%|███████▋                                                                       | 179/1826 [03:48<35:05,  1.28s/it]


KeyboardInterrupt: 

In [180]:
df.to_excel('data/dados.xlsx', index=False)

In [134]:
df.iloc[5]

address                Rua General Chagas Santos - Saúde, São Paulo - SP
url                    https://www.vivareal.com.br/imovel/apartamento...
bedrooms                                                             1.0
bathrooms                                                              1
parking                                                              1.0
amenities              [Ar-condicionado, Churrasqueira, Cozinha, Elev...
title                    Apartamento com 1 Quarto e 1 banheiro para A...
rent                                                              2300.0
Condomínio                                                           506
Preço c/ condomínio                                                 2806
IPTU                                                                 183
Name: 5, dtype: object